In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(32 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [3]:

# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001
num_classes = 10

# Load the CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the CNN
model = CNN(num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data


In [5]:
# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')



Epoch [1/10], Step [100/500], Loss: 1.6472
Epoch [1/10], Step [200/500], Loss: 1.5332
Epoch [1/10], Step [300/500], Loss: 1.3188
Epoch [1/10], Step [400/500], Loss: 1.0930
Epoch [1/10], Step [500/500], Loss: 1.1795
Epoch [2/10], Step [100/500], Loss: 1.1402
Epoch [2/10], Step [200/500], Loss: 1.0532
Epoch [2/10], Step [300/500], Loss: 1.0749
Epoch [2/10], Step [400/500], Loss: 1.0299
Epoch [2/10], Step [500/500], Loss: 0.7932
Epoch [3/10], Step [100/500], Loss: 0.9160
Epoch [3/10], Step [200/500], Loss: 0.8389
Epoch [3/10], Step [300/500], Loss: 0.9102
Epoch [3/10], Step [400/500], Loss: 1.2707
Epoch [3/10], Step [500/500], Loss: 0.9676
Epoch [4/10], Step [100/500], Loss: 0.7784
Epoch [4/10], Step [200/500], Loss: 0.8785
Epoch [4/10], Step [300/500], Loss: 0.8456
Epoch [4/10], Step [400/500], Loss: 0.9161
Epoch [4/10], Step [500/500], Loss: 0.9941
Epoch [5/10], Step [100/500], Loss: 0.7717
Epoch [5/10], Step [200/500], Loss: 0.7209
Epoch [5/10], Step [300/500], Loss: 0.9457
Epoch [5/10

In [6]:
# Testing loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'Test Accuracy of the model on the {total} test images: {(correct / total) * 100:.2f}%')

Test Accuracy of the model on the 10000 test images: 70.83%
